In [1]:
import os

In [2]:
%pwd

'd:\\Kidney-Disease-Classification-DL-MLflow-DVC\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Kidney-Disease-Classification-DL-MLflow-DVC'

In [ ]:
os.environ["MLFLOW_TRACKING_URI"] = f"https://dagshub.com/sujalvishwakarma5/Kidney-Disease-Classification-DL-MLflow-DVC.mlflow"

In [ ]:
import dagshub
dagshub.init(repo_owner='sujalvishwakarma5', repo_name='Kidney-Disease-Classification-DL-MLflow-DVC', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)